# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:


# Loan CSV file generated from WeatherPy Folder
weather_data_baby = "cities.csv"
weather_deez = pd.read_csv(weather_data_baby)
dropna_weather_deez = weather_deez.dropna()
del dropna_weather_deez["Unnamed: 0"]
dropna_weather_deez.head(10)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Leningradskiy,3,RU,1612987269,83,69.3833,178.4167,239.41,4.43
1,Severo-Kuril'sk,100,RU,1612987270,87,50.6789,156.1250,269.61,1.98
2,Kapaa,40,US,1612987075,49,22.0752,-159.3190,296.15,6.69
3,Atuona,1,PF,1612987273,77,-9.8000,-139.0333,299.26,8.33
4,Barrow,1,US,1612987274,68,71.2906,-156.7887,241.15,4.63
5,Marapanim,100,BR,1612987275,91,-0.7175,-47.6997,299.48,2.10
6,Hobart,0,AU,1612987103,82,-42.8794,147.3294,285.15,4.12
7,Jamestown,1,US,1612987040,58,42.0970,-79.2353,269.26,1.54
8,Puerto Ayora,75,EC,1612987085,83,-0.7393,-90.3518,300.15,1.54
9,Punta Arenas,0,CL,1612987083,55,-53.1500,-70.9167,292.15,4.63


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

# Locations
locations = dropna_weather_deez[["Lat", "Lng"]]

humidity = dropna_weather_deez["Humidity"].astype(float)

In [4]:
figdata = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

figdata.add_layer(heat_layer)

#Display figure

figdata



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Filter vacation with zero cloudiness
hotspot_no_cloud = dropna_weather_deez[dropna_weather_deez["Cloudiness"] == 0]
# Filter vacation with max temp above 70 degrees F
hotspot_above_70_degrees = hotspot_no_cloud[hotspot_no_cloud["Max Temp"] > 70]
# Filter vacation with max temp below 80 degrees F
hotspot_below_80_degrees = hotspot_above_70_degrees[hotspot_above_70_degrees["Max Temp"] < 80]
# Filter vacation with wind speed below 10 mph
hotspot_slow_wind = hotspot_below_80_degrees[hotspot_below_80_degrees["Wind Speed"] < 10]
# Filter vacation with humidity below 60 %
hotspot_retire = hotspot_slow_wind[hotspot_slow_wind["Humidity"] < 60]
# Set Index
indexed_hotspot_retire = hotspot_retire.reset_index()
del indexed_hotspot_retire["index"]
indexed_hotspot_retire

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_hotspot_retire["City"])):

    lat = indexed_hotspot_retire.loc[city]["Latitude"]
    lng = indexed_hotspot_retire.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_hotspot_retire["Nearest Hotel"] = hotels
indexed_hotspot_retire

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Nearest Hotel


In [7]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_hotspot_retire.iterrows()]
locations = indexed_hotspot_retire[["Lat", "Lng"]]

In [91]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
figdata.add_layer(markers)

# Display Map
figdata

Figure(layout=FigureLayout(height='420px'))